# Zero-shot prompting

This notebook introduces zero-shot prompting, a powerful technique in prompt engineering that enables language models to perform tasks without being given any examples beforehand. Zero-shot prompting allows AI models to generalize across many different tasks with minimal setup and without the need for training or fine-tuning on specific examples. It’s effective, scalable, and easy to implement.

In [1]:
import os
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Set up OpenAI API key
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')

### Initialize the language model
We instantiate a lightweight GPT model from OpenAI using LangChain.

In [2]:
# Initialize the language model
llm = ChatOpenAI(model="gpt-4o-mini-2024-07-18")

- `ChatOpenAI` from LangChain is used to interact with OpenAI's chat models.
- We use `gpt-4o-mini` for fast and cost-efficient completions.

### Direct task specification
We now define a prompt that explicitly instructs the model to classify text sentiment, without providing any examples. This is a classic zero-shot setup: clear task description but no examples of how to complete it.

In [3]:
# Define the prompt
direct_task_prompt = """Classify the sentiment of the following text as positive, negative, or neutral.
Do not explain your reasoning, just provide the classification.

Text: {text}

Sentiment:"""

# Create a LangChain PromptTemplate from the string
prompt = PromptTemplate.from_template(direct_task_prompt)
# Create a prompt chain by connecting the prompt template to the language model
direct_task_chain = prompt | llm

# Test the direct task specification
texts = [
    "I absolutely loved the movie! The acting was superb.",
    "The weather today is quite typical for this time of year.",
    "I'm disappointed with the service I received at the restaurant."
]

# Loop through the test texts and classify each one
for text in texts:
    result = direct_task_chain.invoke({"text": text}).content  # Pass the input text and get model output
    print(f"Text: {text}")
    print(f"Sentiment: {result}")

Text: I absolutely loved the movie! The acting was superb.
Sentiment: Positive
Text: The weather today is quite typical for this time of year.
Sentiment: Neutral
Text: I'm disappointed with the service I received at the restaurant.
Sentiment: Negative


We will use LangChain’s `PromptTemplate` to define a parameterized prompt, which will be dynamically filled in with different input texts. Then we will pass each completed prompt to the OpenAI model using a LangChain prompt chain.
- The `{text}` placeholder makes the prompt reusable across different inputs.
- `PromptTemplate.from_template(...)`: This converts the raw string into a structured prompt object, enabling variable substitution. In this case, we only have one variable: `text`.
- Prompt Chain (`prompt | llm`): This constructs a LangChain pipeline where the input text is first inserted into the prompt, and the resulting prompt is then passed to the GPT model for generation. This sets up the pipeline: input text -> prompt template -> model.
- We loop through different input texts and classify their sentiment.
- The `invoke` method passes variables to the template and executes the model.

# Format specification
Next, we focus not just on what task the model should perform (writing a news article), but also on how the output should be structured. This type of prompting is known as format specification—an effective zero-shot technique where the prompt clearly defines the format or structure expected in the model’s response. We define a prompt that instructs the model to follow a specific output format: headline, lead, body, and conclusion. This improves consistency, readability, and control, especially when using outputs in downstream applications. This technique is especially useful in real-world applications where structured outputs are necessary—such as generating reports, summaries, blog drafts, or templates for automated communication.

In [4]:
# Define a prompt template
format_spec_prompt = """Generate a short news article about {topic}.
Structure your response in the following format:

Headline: [A catchy headline for the article]

Lead: [A brief introductory paragraph summarizing the key points]

Body: [2-3 short paragraphs providing more details]

Conclusion: [A concluding sentence or call to action]"""

# Create a PromptTemplate from the raw template string
prompt = PromptTemplate.from_template(format_spec_prompt)
# Combine the prompt template with the language model into a chain
format_spec_chain = prompt | llm

# Test the format specification prompting
topic = "The discovery of a new earth-like exoplanet"
# Pass the topic into the chain and get the formatted output
result = format_spec_chain.invoke({"topic": topic}).content
print(result)

**Headline:** Astronomers Unveil New Earth-Like Exoplanet in Habitable Zone

**Lead:** In a groundbreaking discovery, a team of astronomers has identified a new Earth-like exoplanet located within the habitable zone of its star, raising hopes for the potential of extraterrestrial life. Named Kepler-452d, this distant world is reminiscent of our own planet in size and atmospheric conditions, igniting excitement in the scientific community.

**Body:** The exoplanet, located approximately 1,400 light-years away in the constellation Cygnus, is roughly 1.6 times the size of Earth and orbits its sun-like star in a region where temperatures could allow for liquid water to exist. Researchers utilized advanced telescopic technology and data analysis techniques to confirm its existence and assess its potential for supporting life.

Further studies suggest that Kepler-452d has a rocky composition and may possess a similar atmosphere to Earth, making it an ideal candidate for future exploration. T

This prompt not only asks the model to write an article but explicitly defines four sections: `Headline`, `Lead`, `Body`, and `Conclusion`. This communicates clear formatting expectations to the model.
- LangChain PromptTemplate: As in the previous example, `PromptTemplate.from_template(...)` is used to dynamically inject values—in this case, the `{topic}`—into the structured prompt.
- Prompt chain: The chain `prompt | llm` links the formatted prompt to the GPT model, forming a callable pipeline for inference.
- The model receives the full prompt and generates a news-style article, adhering to the requested structure.
  
Providing output format guidelines in the prompt can help structure the AI's response in a zero-shot scenario. By specifying output format, we gain control over not just the content of the model’s response but also its structure, which is often just as important.

### Multi-step reasoning
In this example, we apply a multi-step reasoning prompt—a structured technique that guides the language model to analyze complex content in a systematic way. Instead of asking for a generic response, the prompt explicitly breaks down the task into distinct steps: identifying the main argument, presenting supporting evidence, and considering possible counterarguments. This method enhances the reliability and depth of the model's response, especially for analytical tasks involving nuanced content or critical thinking.

In [5]:
# Define a prompt that guides the model to reason through multiple analytical steps
multi_step_prompt = """Analyze the following text for its main argument, supporting evidence, and potential counterarguments.
Provide your analysis in the following steps:

1. Main Argument: Identify and state the primary claim or thesis.
2. Supporting Evidence: List the key points or evidence used to support the main argument.
3. Potential Counterarguments: Suggest possible objections or alternative viewpoints to the main argument.

Text: {text}

Analysis:"""

# Create a PromptTemplate from the string
prompt = PromptTemplate.from_template(multi_step_prompt)
# Create a chain that links the prompt with the language model
multi_step_chain = prompt | llm

# Test the multi-step reasoning approach
text = """While electric vehicles are often touted as a solution to climate change, their environmental impact is not as straightforward as it seems.
The production of batteries for electric cars requires significant mining operations, which can lead to habitat destruction and water pollution.
Moreover, if the electricity used to charge these vehicles comes from fossil fuel sources, the overall carbon footprint may not be significantly reduced.
However, as renewable energy sources become more prevalent and battery technology improves, electric vehicles could indeed play a crucial role in combating climate change."""

# Invoke the chain with the input text
result = multi_step_chain.invoke({"text": text}).content
print(result)

1. **Main Argument:** The primary claim of the text is that while electric vehicles (EVs) are promoted as a solution to climate change, their overall environmental impact is complex and potentially negative due to the consequences of battery production and the source of electricity used for charging.

2. **Supporting Evidence:**
   - The production of batteries for electric vehicles involves extensive mining operations, which can result in habitat destruction and water pollution.
   - If the electricity used to charge electric vehicles is generated from fossil fuel sources, the reduction in carbon footprint may not be significant.
   - There is potential for electric vehicles to contribute positively to climate change efforts as renewable energy sources become more widely adopted and battery technology advances.

3. **Potential Counterarguments:**
   - Advocates for electric vehicles might argue that, despite the environmental costs of battery production and fossil fuel electricity, EV

The prompt includes clear instructions for the model to follow three specific reasoning steps. By structuring the task this way, we help the model avoid vague or unfocused answers.
- PromptTemplate construction: `PromptTemplate.from_template(...)` dynamically prepares the prompt with a specific `{text}` input that the model will analyze.
- Chain assembly: As before, `prompt | llm` creates a simple and composable LangChain chain that feeds the rendered prompt into the model.
- Input text: The text discusses the pros and cons of electric vehicles and is intentionally complex, making it a good candidate for multi-step analysis.
- Model invocation: The `.invoke(...)` method fills in the `{text}` placeholder with the real input and sends the prompt to the model. The result is a well-structured breakdown of the argument, supporting details, and counterpoints.

By guiding the model through explicit reasoning steps, this technique can significantly improve both the structure and quality of analytical responses—useful for tasks like policy analysis, critical essays, argument mapping, and educational applications. It also reduces ambiguity by clearly framing the model’s thinking process.

## Comparative analysis
We now test two different prompt formats for the same task. Both prompts ask the model to explain the concept of blockchain technology, but they do so in different ways:
- Basic prompt: A minimal instruction—simply ask for an explanation.
- Structured prompt: A more detailed format that instructs the model to break down the explanation into specific subcomponents.

In [6]:
# Define the task for both prompts
task = "Explain conciesly the concept of blockchain technology"

# Define two different prompt templates for comparison
prompt_templates = {
    # Simple one-line instruction
    "Basic": "Explain {task}.",
    # Detailed, step-by-step structure to guide the model
    "Structured": """Explain {task} by addressing the following points:
1. Definition
2. Key features
3. Real-world applications
4. Potential impact on industries"""
}

# Loop through each prompt variant and run the model
for name, template in prompt_templates.items():
    # Create a prompt template from the string
    prompt = PromptTemplate.from_template(template)
    # Combine the prompt with the language model
    chain = prompt | llm
    # Invoke the chain with the input task
    result = chain.invoke({"task": task}).content
    print(f"{name} Prompt Result:\n{result}")
    print("\n" + "-"*50 + "\n")

Basic Prompt Result:
Blockchain technology is a decentralized digital ledger system that records transactions across multiple computers in a way that ensures security, transparency, and immutability. Each transaction is grouped into a block, and these blocks are linked together in chronological order to form a chain. Once a block is added to the chain, altering it becomes nearly impossible without consensus from the network participants. This technology underpins cryptocurrencies like Bitcoin, but it also has applications in various fields, including supply chain management, healthcare, and voting systems, due to its ability to provide secure and verifiable records without a central authority.

--------------------------------------------------

Structured Prompt Result:
### 1. Definition
Blockchain technology is a decentralized digital ledger that records transactions across many computers in a way that the registered transactions cannot be altered retroactively. This ensures transpar

We define a single, consistent task—explaining blockchain technology concisely—so we can objectively compare outputs.
- Two prompt variants:
  - The basic prompt provides minimal structure, leaving it to the model to decide how to present the information.
  - The structured prompt gives explicit headings, which encourages the model to return an organized and comprehensive explanation.
- Each prompt is passed through `PromptTemplate.from_template(...)` to allow the insertion of the `{task}` variable.
- Each prompt is piped (`|`) into the language model, and the `.invoke(...)` method is used to run the prompt.

This comparative analysis helps determine which prompt format is more effective for a given use case. Often, even small changes in prompt phrasing or structure can lead to significantly better results.

This experiment gives us a practical sense of how prompt design influences model behavior—an essential skill when building real-world AI applications.